In [1]:
#5팀
#박세진, 서기욱, 김경영, 양연지
#사용한 영상 : Lenna.png, guide.png(프로그램 안내 이미지)

import numpy as np
import cv2

In [2]:
#최대값 필터
def max_filter(img) :
    height, width= img.shape
    temp, max_out = img.copy(), img.copy()
    
    #3x3 필터를 이용하여 최대값을 구함
    for i in range(height):
        for j in range(width):
            max_out[i, j] = np.max(temp[i:i + 3, j:j + 3])
    
    return max_out

#최소값 필터
def min_filter(img) :
    height, width= img.shape
    temp, min_out = img.copy(), img.copy()
    
    #3x3 필터를 이용하여 최소값을 구함
    for i in range(height):
        for j in range(width):
            min_out[i, j] = np.min(temp[i:i + 3, j:j + 3])
    
    return min_out

#다른 방법의 최소값 최대값 필터
def minmax_filter(image, ksize, mode) :
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.uint8)
    center = ksize // 2
    
    for i in range (center, rows - center) :
        for j in range(center, cols - center) :
            y1, y2 = i - center, i + center + 1
            x1, x2 = j - center, j + center + 1
            mask = image[y1:y2, x1:x2]
            dst[i, j] = cv2.minMaxLoc(mask)[mode]
    
    return dst

#평균값 필터
def average_filter(img, ksize) :
    rows, cols= img.shape[:2]
    dst = np.zeros((rows, cols), np.uint8)
    center = ksize // 2
    
    for i in range(rows) :
        for j in range(cols) :
            y1, y2 = i - center, i + center + 1
            x1, x2 = j - center, j + center + 1
            
            if y1 < 0 or y2 > rows or x1 < 0 or x2 > cols :
                dst[i, j] = img[i, j]
            else :
                mask = img[y1:y2, x1:x2]
                dst[i, j] = cv2.mean(mask)[0]
    return dst

In [3]:
img = cv2.imread('./images/Lenna.png', cv2.IMREAD_GRAYSCALE)
guide = cv2.imread('./images/guide.png', cv2.IMREAD_GRAYSCALE)

click = False     # Mouse 클릭된 상태  False == 클릭 안된 상태 / True == 클릭된 상태
start_x, start_y = -1, -1 # 마우스를 누른 상태에서 x,y 좌표 저장
end_x, end_y = -1, -1 # 마우스를 때면 x,y 좌표 저장
key = 'a' #마우스 클릭시 필터링할 키값
power = 15 # 축소 후 확대의 배수

def draw_rectangle(event, x, y, flags, param):
    global end_x, end_y, start_x, start_y, click, key, power                           # 전역변수 사용

    if event == cv2.EVENT_LBUTTONDOWN:                      
        click = True 
        start_x, start_y = x,y #마우스 클릭 시 좌표 저장
    elif event == cv2.EVENT_LBUTTONUP:
        click = False                                 
        end_x, end_y = x, y #마우스 클릭 해제 시 좌표 저장

        #만약 끝좌표가 시작좌표보다 작으면 두 좌표를 바꿔줌
        if start_x > end_x :
            start_x, end_x = end_x, start_x
            
        if start_y > end_y :
            start_y, end_y = end_y, start_y
            
        clipImg = img[start_y:end_y, start_x:end_x]
        
        #평균값 필터
        if key == 'a' :
            clipImg = average_filter(clipImg, 5)
            
        #최대값 필터
        elif key == 'x' :
            clipImg = max_filter(clipImg)
        
        #최소값 필터
        elif key == 'm' :
            clipImg = min_filter(clipImg)
        
        #축소 후 확대
        elif key == 'l' :
            #만약 클립된 이미지의 크기가 power의 배수가 아니라면 power의 배수가 될 수 있게 값을 더해줌
            if (end_x - start_x) % power != 0:
                end_x += power - (end_x - start_x) % power
            
            if (end_y - start_y) % power != 0:
                end_y += power - (end_y - start_y) % power
                
            clipImg = img[start_y:end_y, start_x:end_x]
            
            #축소 후 확대
            clipImg = cv2.resize(clipImg, None, fx = 1/power, fy = 1/power, interpolation = cv2.INTER_CUBIC)
            clipImg = cv2.resize(clipImg, None, fx = power, fy = power, interpolation = cv2.INTER_CUBIC)
        
        #클립된 이미지를 원래 이미지에 붙여넣기
        img[start_y:end_y, start_x:end_x] = clipImg

#마우스 이벤트를 받기 위한 함수
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_rectangle)    

cv2.imshow('guide', guide)

while True:
    cv2.imshow('image', img)    
    
    k = cv2.waitKey(1) & 0xFF   
    
    #키값에 따라 다른 필터 적용
    if k == 27:               
        break
    elif k == ord('a'): 
        key = 'a'
        print(key)
    elif k == ord('x') :
        key = 'x'
        print(key)
    elif k == ord('m') :
        key = 'm'
        print(key)
    elif k == ord('l') :
        key = 'l'
        print(key)
          
cv2.destroyAllWindows()

l
